In [1]:
import pandas as pd
import pickle
import numpy as np
from Constants import Const
import torch
import torch.nn as nn
import cv2
from pointcloud_utils import *
import open3d as o3d

In [2]:
with open(Const.data_dir+'processed_dicoms.p','rb') as f:
    plist = pickle.load( f ) 
plist[-1]

{'id': 5,
 'Spacing': (0.9375, 0.9375, 3.0),
 'Volumes': array([86.53126863,  5.81267612, 29.51884323,  8.69137688,  3.98198162,
         3.0224147 , 18.70934397,  4.91059477,  4.95923641,  5.07641855,
         1.07674906,  0.55937887, 26.83470903, 25.78228079,  1.68919154,
         0.        , 13.65724625, 12.99395114,  7.00881829, 47.24872143,
        49.31156921]),
 'ArrayDicom': array([[[-1000.,  -850.,  -823., ...,  -816.,  -861.,  -824.],
         [ -840.,  -837.,  -859., ...,  -859.,  -822.,  -819.],
         [ -833.,  -852.,  -834., ...,  -822.,  -838.,  -819.],
         ...,
         [ -893.,  -883.,  -882., ...,  -826.,  -838.,  -833.],
         [ -882.,  -876.,  -901., ...,  -834.,  -822.,  -831.],
         [ -873.,  -888.,  -905., ...,  -818.,  -829.,  -836.]],
 
        [[-1000.,  -807.,  -844., ...,  -849.,  -863.,  -872.],
         [ -814.,  -832.,  -825., ...,  -875.,  -856.,  -856.],
         [ -844.,  -811.,  -799., ...,  -853.,  -855.,  -862.],
         ...,
        

In [49]:
def vectorize_contours(pdict,rois=None,min_size = 2):
    contours = pdict['contours']
    contours = pcloud_to_concave_hull(contours)
    if rois is None:
        rois = sorted(contours.keys())
    array = []
    for roi in rois:
        contour = contours.get(roi)
        if contour is None or contour.shape[0] < min_size:
            contour = np.zeros((min_size,3))
        array.append(contour)
    #returns a list of variable size pointclouds
    return array
        
def downsample_pcloud(xyz,size=2):
#     print(xyz.shape)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd2 = pcd.voxel_down_sample(size)
    xyz_new = np.asarray(pcd2.points)
#     print(xyz_new.shape[0],xyz_new.shape[0]/xyz.shape[0],1/size)
    return xyz_new

def resize_pcloud(xyz, npoints = 2000,jitter=.1):
    #voxel downsample
    if xyz.shape[0] < 2:
        return np.zeros((npoints,3))
    correction = 0
    prev_size = xyz.shape[0]
    while xyz.shape[0] > npoints:
        xyz = downsample_pcloud(xyz,(xyz.shape[0]/npoints)+correction)
        if xyz.shape[0] >= prev_size:
            correction += .1
        prev_size = xyz.shape[0]
#         print('---')
#     print(xyz.shape)
    #padd pointcloud with the centroid of the point + some jitter
    diff = npoints - xyz.shape[0]
    pad = np.zeros((diff,3))
    pad[:] = cloud_centroid(xyz)
    if jitter:
        pad = pad + (np.random.random(pad.shape) - .5)*jitter
    new_points = np.vstack([xyz,pad])
#     print('new points',new_points.shape)
#     print('------------------------')
    return new_points


def vectorize_plist_pclouds(plist,rois=None,size=1000):
    #how to make htis a uniform shape
    arrays = []
    if rois is None:
        rois = list(plist[0]['roi_mask_map'].values())
    for patient in plist:
        arr = vectorize_contours(patient,rois=rois)
        print(arr[0].shape)
        if size is not None and size > 0:
            arr = [resize_pcloud(a,size) for a in arr]
            arr = np.stack(arr)
        arrays.append(arr)
    if size is not None and size > 0:
        arrays = np.stack(arrays)
    return arrays
    
test = vectorize_plist_pclouds(plist[2:])
test.shape

(3997, 3)
(2, 3)
(10464, 3)


(3, 21, 1000, 3)

In [57]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.graph_objects as go

def plot_pcloud_array(array,figsize=(8,8)):
    fig = plt.figure(figsize=figsize,facecolor='w')
    ax = fig.add_subplot(projection='3d')
    ax.set_facecolor('w')
    ax.grid(False)
    colors = mpl.colormaps['tab20']
    assert(array.ndim == 3)
    assert(array.shape[-1] == 3)
    for pointcloud in array:
        size = .2
        ax.scatter(pointcloud[:,0],pointcloud[:,1],pointcloud[:,2],s=size,color='grey')
    return ax

def plotly_pcloud_array(array):
    data = []
    assert(array.ndim == 3)
    assert(array.shape[-1] == 3)
    for points in array:
        if points.shape[0] < 5:
            continue
        color='gray'
        subplot = go.Scatter3d(
            x=points[:,0], y=points[:,1], z=points[:,2], 
            mode='markers',
            marker=dict(size=1.5,color=color)
        )
        data.append(subplot)
    fig = go.Figure(
        data = data,
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        )
    )
    return fig

plotly_pcloud_array(test[0])

In [ ]:
#code I tried to get convolutional network to work on

In [ ]:
def get_array(patient):
    #extracts an array of image slices with contours
    values = patient['ArrayDicom']
    mask = patient['mask']
    
    #make stuff that isn't a contour 0
    good_values = values * (mask > 0)
    
    #trim slices with no contours
    not_empty = good_values.sum(axis=1).sum(axis=1) > 0.01
    first_true = -1
    last_true = -1
    for pos,boolean in enumerate(not_empty):
        if boolean:
            if first_true < 0:
                first_true = pos
            else:
                last_true = max(last_true,pos)
    return good_values[first_true:last_true+1]

def pad_height(array3d,height,fill=0):
    shape = array3d.shape[1:]
    npads=height- array3d.shape[0] 
    assert(npads >= 0)
    if npads == 0:
        return array3d
    
    pad = np.full((npads,shape[0],shape[1]),fill)
    return np.vstack([array3d,pad])

def arrays_from_plist(plist, pad=0,scale= True, add_channel=False,square=True,img_size=128):
    #returns an array of image vlaues n_patients x n_images x width x height
    x = [get_array(p) for p in plist]
    if square:
        max_height = img_size
    else:
        max_height = np.max([xx.shape[0] for xx in x ])
    x = [pad_height(xx,max_height,pad) for xx in x]
    #make it square
    if img_size is not None:
        x = [np.stack([cv2.resize(xx, (img_size,img_size)) for xxx in xx]) for xx in x]
    x = np.stack(x)
    #scale to be 0-1
    if scale:
        x = (x-x.min())/(x.max() - x.min())
    #adds a dimension so  htat the model fits for a 3d convolution for pytorch
    if add_channel:
        s = x.shape
        x = x.reshape((s[0],1,s[1],s[2],s[3]))
    return x

arrays_from_plist(plist,add_channel=True).shape

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable
from functools import partial


class C3Encoder(nn.Module):
    def __init__(self,
                 hidden_dims = [4,8,16],
                 fc_dims=[64,32], #last fc_dim will be 
                 conv_kernel_size=3,
                 channels=1,
                ):

        super(C3Encoder, self).__init__()
        groups = []
        curr_in = channels
        for hidden_dim in hidden_dims:
            conv = self.make_conv_group(curr_in,hidden_dim,conv_kernel_size)
            curr_in = hidden_dim
            groups.append(conv)
        self.groups= nn.ModuleList(groups)
        
        self.init_fc_layers(fc_dims)
        
        
    def make_conv_group(self,curr_in, curr_out,kernel_size):
        conv = nn.Sequential(
            nn.Conv3d(curr_in,curr_out,kernel_size=kernel_size,padding=1),
            nn.BatchNorm3d(curr_out),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(1, 2, 2)),
        )
        return conv
        

    def init_fc_layers(self,fc_dims):
        fc_layers = []
        for fc_dim in fc_dims:
            group = self.make_linear_group(fc_dim)
            fc_layers.append(group)
            
        self.fc_layers = nn.ModuleList(fc_layers)
        self.output_size = fc_dims[-1]
        
    def make_linear_group(self, curr_out,**kwargs):
        return nn.LazyLinear(curr_out,**kwargs)
    
    def forward(self, x):
        x = torch.from_numpy(x).float().cuda()
        for group in self.groups:
            x = group(x)
        out = x.view(x.size(0), -1)
        for layer in self.fc_layers:
            out = layer(out)
        return out
    

    
class C3Decoder(nn.Module):
    def __init__(self,
                fc_dims=[16,16],
                hidden_dims =[32,62],
                 kernel_sizes=[3,2],
                 image_output_size = [104,128,128],
                ):
        super(C3Decoder,self).__init__()
        fc = fc_dims + [image_output_size[0]]#the last full-connected will be the size of the stack of images
        #so I can use it as the equivalent of the channels section (this is a bad idea)
        self.init_fc_layers(fc_dims)
        
        conv_layers = []
        curr_in = fc_dims[-1]
        for dim,kernel_size in zip(hidden_dims,kernel_sizes):
            layer = self.dconv_block(curr_in,dim,kernel_size) #nn.ConvTranspose3d(curr_in,dim,kernel_size)
            conv_layers.append(layer)
            curr_in = dim
        self.conv_layers = nn.ModuleList(conv_layers)
        
        #have to convert nxd to nxchannelsxslicesxhieghtxwidth
        #temp
        self.orient = lambda x: x.view(x.shape[0],-1,1,1,1)
        
        #calculate how big the final output will be
        #each kernel adds last three dimensions as kernel_size - 1, assume we start at one
        #final hidden dims is # of channels
        final_size = hidden_dims[-1] * (np.sum([ks - 1 for ks in kernel_sizes]) + 1)
        target_size = np.prod(image_output_size)
        print(final_size, target_size)
        
    def deconv_block(self,in_dims,out_dims,ksize):
        deconv = nn.Sequential(
            nn.ConvTranspose3d(in_dims,out_dims,ksize),
            nn.BatchNorm3d(num_features=out_dims),
            nn.ReLU(),
        )
        return deconv
    
    def init_fc_layers(self,fc_dims):
        fc_layers = []
        for fc_dim in fc_dims:
            group = self.make_linear_group(fc_dim)
            fc_layers.append(group)
            
        self.fc_layers = nn.ModuleList(fc_layers)
        self.output_size = fc_dims[-1]
        
    def make_linear_group(self, curr_out,**kwargs):
        return nn.LazyLinear(curr_out,**kwargs)
    
    def forward(self,x):
        print(x.shape)
        for layer in self.fc_layers:
            x = layer(x)
        print(x.shape)
        x = self.orient(x)
        print('after orient', x.shape)
        for layer in self.conv_layers:
            x = layer(x)
            print(x.shape)
        return x
    
class C3AutoEncoder(nn.Module):
    
    def __init__(self,
                 encoder=None,
                 decoder=None,
                ):
        super(C3AutoEncoder,self).__init__()
        if encoder is None:
            encoder = C3Encoder()
        if decoder is None:
            decoder = C3Decoder()
            
        self.encoder=encoder
        self.decoder=decoder
        
    def forward(self,x):
        x = self.encoder(x)
        print('deocder')
        x = self.decoder(x)
        return x
    
torch.cuda.empty_cache()
# testx = arrays_from_plist(plist,add_channel=True,resize=128)
test = C3AutoEncoder().cuda()
test
# test( testx[1:3] ).shape

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.device('cuda')